In [25]:
from gurobipy import GRB, quicksum
import gurobipy as gb
import pandas as pd

players_df = pd.read_csv('/Users/changlu/Downloads/BasketballPlayers.csv')

# Create the optimization model
model = gb.Model("Basketball Team Selection")

# Decision variables: One for each player
x = model.addVars(players_df.index, vtype=GRB.BINARY, name="Player")

# Objective function: Maximize the sum of selected players' skills
skills = ['Ball Handling', 'Shooting', 'Rebounding', 'Defense', 'Athletic Ability', 'Toughness', 'Mental Acuity']
model.setObjective(quicksum(x[i] * sum(players_df.loc[i, skill] for skill in skills) for i in players_df.index), GRB.MAXIMIZE)

# Total number of players invited exactly 21
model.addConstr(quicksum(x[i] for i in players_df.index) == 21, "TotalPlayers21")

# Skill averages - Assuming the requirement is an average of at least 2.05 for each skill
for skill in skills:
    model.addConstr(quicksum(x[i] * players_df.loc[i, skill] for i in players_df.index) / 21 >= 2.05, f"Avg{skill}")

# Positional requirements - Adjusted to ensure a balanced team composition
guards = players_df[players_df['Position'].isin(['G', 'G/F'])].index
forwards_centers = players_df[players_df['Position'].isin(['F', 'C', 'F/C'])].index
model.addConstr(quicksum(x[i] for i in guards) >= 0.3 * 21, "MinGuards")
model.addConstr(quicksum(x[i] for i in forwards_centers) >= 0.4 * 21, "MinForwardsCenters")

# Mutual Exclusivity - Correctly defined based on player groups
group1 = players_df[(players_df['Number'] >= 20) & (players_df['Number'] <= 24)].index
group2 = players_df[(players_df['Number'] >= 72) & (players_df['Number'] <= 78)].index
model.addConstr(quicksum(x[i] for i in group1) + quicksum(x[i] for i in group2) <= 1, "MutualExclusivity")

# Conditional Inclusion - Correctly implemented based on problem specifics
group3 = players_df[(players_df['Number'] >= 105) & (players_df['Number'] <= 114)].index
group4 = players_df[(players_df['Number'] >= 45) & (players_df['Number'] <= 49)].index
group5 = players_df[(players_df['Number'] >= 65) & (players_df['Number'] <= 69)].index

# If any player from 105-114 is selected, at least one player from 45-49 must be invited.
for i in range(105, 115):
    model.addConstr(x[i] <= sum(x[j] for j in range(45, 50)), "AtLeastOne_45_49_If_105_114")

# If any player from 105-114 is selected, at least one player from 65-69 must be invited.
for i in range(105, 115):
    model.addConstr(x[i] <= sum(x[j] for j in range(65, 70)), "AtLeastOne_65_69_If_105_114")
    
# Representation from Various Ranges - Ensures diversity in player selection
for start in range(1, 151, 10):
    range_group = players_df[(players_df['Number'] >= start) & (players_df['Number'] < start+10)].index
    model.addConstr(quicksum(x[i] for i in range_group) >= 1, f"Rep_{start}-{start+9}")

# Solve the model
model.optimize()

# Print selected players
for v in model.getVars():
    if v.x > 0.5:
        print(f"{v.VarName} = {v.x}")

# Check the model's status
print("Model Status: ", model.Status)




Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 22.4.0 22E261)

CPU model: Intel(R) Core(TM) i5-1030NG7 CPU @ 1.10GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 46 rows, 150 columns and 1632 nonzeros
Model fingerprint: 0x7656062d
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s
Presolved: 46 rows, 149 columns, 1284 nonzeros
Variable types: 0 continuous, 149 integer (149 binary)

Root relaxation: objective 3.610000e+02, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     361.0000000  361.00000  0.00%     -    0s

E

In [26]:
players_df.head()

,Number,Position,Ball Handling,Shooting,Rebounding,Defense,Athletic Ability,Toughness,Mental Acuity
0,1,G/F,1,2,3,2,1,2,1
1,2,G/F,1,1,1,2,3,2,3
2,3,G/F,3,1,1,2,3,2,1
3,4,G/F,2,3,2,2,2,1,1
4,5,F/C,1,2,3,3,3,3,2


# g

In [27]:
# Simulating player data
players_df = pd.DataFrame({
    'Number': range(1, 151),
    'Position': ['G', 'F', 'C', 'G/F', 'F/C'] * 30  # Example positions for players
})

# Simulated selected players based on the example provided earlier
selected_player_ids = [6, 10, 21, 36, 46, 55, 66, 73, 75, 89, 94, 103, 109, 110, 127, 131, 132, 133, 140, 143, 147]

# Count how many guards were invited
guards_invited = players_df[(players_df['Number'].isin(selected_player_ids)) & (players_df['Position'].isin(['G', 'G/F']))].shape[0]

guards_invited

9

# h

In [28]:
# Initial number of invitations
num_invitations = 21

# Placeholder for the smallest number of invitations that results in an infeasible solution
smallest_infeasible_invitations = num_invitations

# Placeholder for infeasible constraints
infeasible_constraints = []

# Attempt to find the smallest number of invitations
while True:
    # Reset the model
    model = gb.Model("Basketball Team Selection")

    # Decision variables: One for each player
    x = model.addVars(players_df.index, vtype=GRB.BINARY, name="Player")

    # Objective function: Maximize the sum of selected players' skills
    skills = ['Ball Handling', 'Shooting', 'Rebounding', 'Defense', 'Athletic Ability', 'Toughness', 'Mental Acuity']
    model.setObjective(quicksum(x[i] * sum(players_df.loc[i, skill] for skill in skills) for i in players_df.index), GRB.MAXIMIZE)

    # Constraints

    # Total number of players invited
    model.addConstr(quicksum(x[i] for i in players_df.index) == num_invitations, "TotalPlayers")

    # Skill averages
    for skill in skills:
        model.addConstr(quicksum(x[i] * players_df.loc[i, skill] for i in players_df.index) / num_invitations >= 2.05, f"Avg{skill}")

    # Positional requirements
    guards = players_df[players_df['Position'].isin(['G', 'G/F'])].index
    forwards_centers = players_df[players_df['Position'].isin(['F', 'C', 'F/C'])].index
    model.addConstr(quicksum(x[i] for i in guards) >= 0.3 * num_invitations, "MinGuards")
    model.addConstr(quicksum(x[i] for i in forwards_centers) >= 0.4 * num_invitations, "MinForwardsCenters")

    # Mutual Exclusivity
    group1 = players_df[(players_df['Number'] >= 20) & (players_df['Number'] <= 24)].index
    group2 = players_df[(players_df['Number'] >= 72) & (players_df['Number'] <= 78)].index
    model.addConstr(quicksum(x[i] for i in group1) + quicksum(x[i] for i in group2) <= len(group1), "MutualExclusivity")

    # Conditional Inclusion
    group3 = players_df[(players_df['Number'] >= 105) & (players_df['Number'] <= 114)].index
    group4 = players_df[(players_df['Number'] >= 45) & (players_df['Number'] <= 49)].index
    group5 = players_df[(players_df['Number'] >= 65) & (players_df['Number'] <= 69)].index
    model.addConstrs((x[i] <= quicksum(x[j] for j in group4) for i in group3), "CondInclusion1")
    model.addConstrs((x[i] <= quicksum(x[j] for j in group5) for i in group3), "CondInclusion2")

    # Representation from Various Ranges
    for start in range(1, 151, 10):
        model.addConstr(quicksum(x[i] for i in players_df[(players_df['Number'] >= start) & (players_df['Number'] < start+10)].index) >= 1, f"Rep_{start}-{start+9}")

    # Solve the model
    model.optimize()

    # If the model is infeasible, break and record the current number of invitations
    if model.Status == GRB.INFEASIBLE:
        smallest_infeasible_invitations = num_invitations
        break
    else:
        # Decrease the number of invitations for the next iteration
        num_invitations -= 1

smallest_infeasible_invitations

KeyError: 'Ball Handling'